<div style="text-align: center;">
    <span style="color: #000000; font-size: 30px; font-weight: bold; font-family: Arial;  margin: 50px; ">
        Final Assignment Part Two
    </span>
</div>

#### <centre>form an optimal portfolio and reblance it</centre>


Choose a collection of assets. Using simulation, find the weights that will lead to the minimum variance portfolio. Make a python function that will purchase the correct amount of assets on Alpaca.

## Step 0  import packages and set parameters

In [1]:
pip install alpaca-trade-api

In [2]:
import yfinance as yf
import numpy as np
from scipy.optimize import minimize
import requests
import json
import alpaca_trade_api as tradeapi

# Choose a collection of assets

In [19]:
tickers = ['SLB',  'OXY', 'BP',  'WMB',  'BKR', 'APA', 'HES']

In [18]:
np.sum(np.array([1,2,3]))

6

# Step 1 Get Optimal Weights


In [20]:
# 获取股票数据
data = yf.download(tickers, start='2000-01-01', end='2023-09-07')['Adj Close']

# 计算每只股票的日收益率
returns = data.pct_change().dropna()

# 计算协方差矩阵
cov_matrix = returns.cov()

# 定义目标函数，最小化方差
def objective(weights):
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    return portfolio_variance

# 设置约束条件，这里假设权重之和为1
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# 设置初始权重猜测值
#一开始我是用的initial_weights = np.ones(len(tickers)) / len(tickers)
initial_weights = np.random.rand(len(tickers))
initial_weights /= initial_weights.sum() 

# 最小化目标函数，使用不同的优化方法
result = minimize(objective, initial_weights, constraints=constraints, method='trust-constr')


# 最优权重
optimal_weights = result.x

print("Optimal weights：", optimal_weights)
print("minimum variance：", result.fun)


[*********************100%%**********************]  7 of 7 completed


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Optimal weights： [0.18095087 0.11210211 0.17463791 0.03602531 0.21230241 0.11132801
 0.17265339]
minimum variance： 0.00046714098736235643


# Step 2 Get Account Balance

In [21]:
#与羊驼关联
API_KEY = "PKOK1AECOYSVDXHMQ7ON"
SECRET_KEY = "xeUuUHj4TuLJ95NaSSMbdKRWERyPGJofYbdoioki"
BASE_URL = "https://paper-api.alpaca.markets"
ACCOUNT_URL = BASE_URL + "/v2/account"
ORDERS_URL =  BASE_URL + "/v2/orders"

In [22]:
r = requests.get(ACCOUNT_URL, headers = {'APCA-API-KEY-ID': API_KEY,
                                         'APCA-API-SECRET-KEY':  SECRET_KEY})
info = json.loads(r.content)
accountval = float(info["cash"])

# Step 3 Get number of Shares

In [29]:
# 计算每个股票的股票数量
shares = []
prices = []
for i, symbol in enumerate(tickers):
    weight = optimal_weights[i]
    price = data[symbol][-1]  # 获取最后一个交易日的股价
    qty = (weight * accountval) / price
    qty = int(qty)  # 取整数部分
    shares.append(qty)

    prices.append(price)




for i, symbol in enumerate(tickers):
    print(f"The optimal number of stock {symbol} : {shares[i]}")

The optimal number of stock SLB : -1644
The optimal number of stock OXY : -950
The optimal number of stock BP : -2493
The optimal number of stock WMB : -598
The optimal number of stock BKR : -3162
The optimal number of stock APA : -1382
The optimal number of stock HES : -608


In [31]:
np.sum(np.array(shares)*np.array(prices))




-554268.3919754028

In [ ]:
sum = np.array(shares)*np.array(prices)

sum,np.sum(sum)

[-963, -732, -599, -2183, -3398, -2904, -605]

KeyboardInterrupt: 

KeyboardInterrupt: 

KeyboardInterrupt: 

# Step 4 Purchase amounts

In [49]:
def create_order(symbol, qty, side, type_, time_in_force):
    data = {
            "symbol": symbol,
            "qty": qty,
            "side": side,
            "type": type_,
            "time_in_force": time_in_force
            }
    r = requests.post(ORDERS_URL, json=data, headers = {'APCA-API-KEY-ID': API_KEY, 
                                                        'APCA-API-SECRET-KEY':  SECRET_KEY})

    return json.loads(r.content)


# REBALANCE PORTFOLIO TO CHECK THAT THE WEIGHTS ARE ALIGNED EACH DAY

In [46]:
# 计算当前股票数量
current_shares = []
for i, symbol in enumerate(tickers):
    latest_prices = data.iloc[-1]
    price = latest_prices[symbol]
    qty = (accountval / len(tickers)) / price
    qty = int(qty)  # 取整数部分
    current_shares.append(qty)

# 打印当前股票数量
for i, symbol in enumerate(tickers):
    print(f"The current number of stock {symbol} : {current_shares[i]}")

# 计算最优股票数量与当前股票数量之间的差异
difference = [shares[i] - current_shares[i] for i in range(len(tickers))]

# 打印差异
for i, symbol in enumerate(tickers):
    print(f"The difference of stock {symbol} : {difference[i]}")

The current number of stock SLB : 2341
The current number of stock OXY : 2184
The current number of stock BP : 3679
The current number of stock WMB : 4277
The current number of stock BKR : 3838
The current number of stock APA : 3199
The current number of stock HES : 905
The difference of stock SLB : 2034
The difference of stock OXY : 626
The difference of stock BP : -2550
The difference of stock WMB : 1977
The difference of stock BKR : -1238
The difference of stock APA : -2898
The difference of stock HES : 275


In [48]:
# Execute Trades to accoujnt for the difference
for i, symbol in enumerate(tickers):
    qty_diff = difference[i]
    if qty_diff < 0:
        # 卖出股票
        create_order(symbol, abs(qty_diff), "sell", "market", "day")
# 买入操作
for i, symbol in enumerate(tickers):
    qty_diff = difference[i]
    if qty_diff > 0:
        # 买入股票
        create_order(symbol, qty_diff, "buy", "market", "day")

In [ ]:
create_order('APA',0.5,"sell","market","day")


In [36]:
BASE_URL = "https://paper-api.alpaca.markets"
time_url = BASE_URL+"/v2/calendar"

api = tradeapi.REST(key_id=API_KEY,secret_key=SECRET_KEY,base_url=BASE_URL)

import datetime

# Check if the market is open now.
clock = api.get_clock()
print('The market is {}'.format('open.' if clock.is_open else 'closed.'))

# Check when the market was open on Dec. 1, 2018
date = "2023-09-15"
calendar = api.get_calendar(start=date, end=date)[0]
print('The market opened at {} and closed at {} on {}.'.format(
    calendar.open,
    calendar.close,
    date
))

The market is closed.
The market opened at 09:30:00 and closed at 16:00:00 on 2023-09-15.
